## Testing Locally

In [10]:
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_WIDTH = 128


def normalize_img(image):
    """Normalizes image.

    * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
    * Casts values from `uint8` to `float32`
    * Scales values from [0, 255] to [0, 1]

    Returns:
      A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
    """
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.


def normalize_img_and_label(image, label):
    """Normalizes image and label.

    * Performs normalize_img on image
    * Passes through label unchanged

    Returns:
      Tuple (image, label) where
      * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
        channels)
      * label is an unchanged integer [0, 4] representing flower type
    """
    return normalize_img(image), label

output_directory = "gs://vtxmodels-demos/tensorflow/tboard-demo/"

logging.info('Loading and preprocessing data ...')
dataset = tfds.load('tf_flowers:3.*.*',
                    split='train',
                    try_gcs=True,
                    shuffle_files=True,
                    as_supervised=True)
dataset = dataset.map(normalize_img_and_label,
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(128)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

logging.info('Creating and training model ...')
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(5)  # 5 classes
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir="gs://vtxdemos/models/tensorflow/tboard/logs",
  histogram_freq=1)
model.fit(dataset, epochs=10, callbacks=[tensorboard_callback])

Epoch 1/10
29/29 [==============================] - ETA: 0s - loss: 1.7003 - accuracy: 0.2790

2023-09-06 14:55:35.198826: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2013265920 exceeds 10% of free system memory.


29/29 [==============================] - 43s 1s/step - loss: 1.7003 - accuracy: 0.2790
Epoch 2/10
29/29 [==============================] - ETA: 0s - loss: 1.2354 - accuracy: 0.4856

2023-09-06 14:56:10.182562: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2013265920 exceeds 10% of free system memory.


29/29 [==============================] - 36s 1s/step - loss: 1.2354 - accuracy: 0.4856
Epoch 3/10
29/29 [==============================] - ETA: 0s - loss: 1.0519 - accuracy: 0.5787

...

29/29 [==============================] - 30s 1s/step - loss: 1.0519 - accuracy: 0.5787
Epoch 4/10
29/29 [==============================] - ETA: 0s - loss: 0.9224 - accuracy: 0.6403

2023-09-06 14:57:25.387819: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2013265920 exceeds 10% of free system memory.


29/29 [==============================] - 39s 1s/step - loss: 0.9224 - accuracy: 0.6403
Epoch 5/10
29/29 [==============================] - ETA: 0s - loss: 0.8391 - accuracy: 0.6804

2023-09-06 14:57:55.537147: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2013265920 exceeds 10% of free system memory.


29/29 [==============================] - 30s 1s/step - loss: 0.8391 - accuracy: 0.6804
Epoch 6/10
29/29 [==============================] - 39s 1s/step - loss: 0.7661 - accuracy: 0.7153
Epoch 7/10
29/29 [==============================] - 34s 1s/step - loss: 0.6351 - accuracy: 0.7616
Epoch 8/10
29/29 [==============================] - 30s 1s/step - loss: 0.5202 - accuracy: 0.8174
Epoch 9/10
29/29 [==============================] - 39s 1s/step - loss: 0.4548 - accuracy: 0.8414
Epoch 10/10
29/29 [==============================] - 30s 1s/step - loss: 0.3590 - accuracy: 0.8777


In [4]:
logging.info(f'Exporting SavedModel to: {output_directory}')
# Add softmax layer for intepretability
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model.save("gs://vtxdemos/models/tensorflow/tboard")

INFO:tensorflow:Assets written to: gs://vtxdemos/models/tensorflow/tboard/assets


INFO:tensorflow:Assets written to: gs://vtxdemos/models/tensorflow/tboard/assets


## Uploading Log to Experiment/tensorboard

In [11]:
from google.cloud import aiplatform

aiplatform.start_upload_tb_log(
    tensorboard_id="6521344201827811328",
    tensorboard_experiment_name="tensorflow-tboard-1",
    logdir="gs://vtxdemos/models/tensorflow/tboard/logs",
    experiment_display_name="tensorflow-tboard-1",
    run_name_prefix="tensorflow1-",
    description="experiment",
)
aiplatform.end_upload_tb_log()

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+254356041555+locations+us-central1+tensorboards+6521344201827811328+experiments+tensorflow-tboard-1
